In [1]:
import pandas as pd
import os
import numpy as np

import Utils as ut
import CortesAlignmentFile as ca
import mySampler as ms
 
from sklearn.model_selection import StratifiedShuffleSplit

## Dataset Configuration - Toy

In [2]:
from sklearn import datasets
# make_classification(n_samples=100, n_features=20, n_informative=2, n_redundant=2, n_repeated=0, 
#                     n_classes=2, n_clusters_per_class=2, weights=None, flip_y=0.01, class_sep=1.0, 
#                     hypercube=True, shift=0.0, scale=1.0, shuffle=True, random_state=None)
ds, label = datasets.make_classification(n_samples=300, n_features=30, n_informative=10, n_redundant=0, n_classes=2)
ds_names = ['ds1', 'ds2', 'ds3']

In [3]:
np.where(label==0)
label[np.where(label==0)]=-1

In [4]:
def random_sampling(X, y, n_samples):
    from sklearn.model_selection import StratifiedShuffleSplit
    return next(StratifiedShuffleSplit(n_splits=1, test_size=n_samples).split(X, y))

In [5]:
tr_idx, ts_idx = random_sampling(ds, label, n_samples=0.25)

ds1 = np.hstack([ds[:,:5], ds[:,10:15]])
ds1_tr = ds1[tr_idx]
ds1_ts = ds1[ts_idx]

ds2 = np.hstack([ds[:,5:10], ds[:,15:20]])
ds2_tr = ds2[tr_idx]
ds2_ts = ds2[ts_idx]

ds3 = ds[:,20:]
ds3_tr = ds3[tr_idx]
ds3_ts = ds3[ts_idx]

y_c_ = label[tr_idx]
y_c_test = label[ts_idx]

ds_list_complete = [ds1, ds2, ds3]
ds_list = [ds1_tr, ds2_tr, ds3_tr]
ds_test = [ds1_ts, ds2_ts, ds3_ts]

## Kernel Definition

In [6]:
kernel_names_1 = ['laplacian', 'polynomial', 'gaussian']
kernel_type_1 = [{'laplacian':[0.1, 0.2, 0.3], 'polynomial':[2, 3], 'gaussian':[0.2, 0.3]},
               {'laplacian':[0.5, 0.7], 'polynomial':[5, 8], 'gaussian':[0.5, 0.7]}]

In [7]:
kernel_names_2 = ['sigmoid', 'polynomial', 'gaussian']
kernel_type_2 = [{'sigmoid':[0.1, 0.2, 0.3], 'polynomial':[2, 3], 'gaussian':[0.2, 0.3]},
               {'sigmoid':[0.5, 0.7], 'polynomial':[5, 8], 'gaussian':[0.5, 0.7]}]

In [20]:
kernel_names_3 = ['linear', 'polynomial', 'gaussian']
kernel_type_3 = [{'linear':[0.5], 'polynomial':[2, 3, 7], 'gaussian':[0.1, 0.5, 0.7]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}]

In [9]:
estimator = ca.centeredKernelAlignment

## Basic approach

In [10]:
sampler = ms.mySampler(n_splits=3, test_size=.25)

In [11]:
result1 = sampler.sample(kernel_type_1, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)


1 split out of 3 ...
	Working on config 1 of 2: {'laplacian': [0.1, 0.2, 0.3], 'polynomial': [2, 3], 'gaussian': [0.2, 0.3]}
Fold no. 1
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
		Perfomances computed for 800
		Perfomances computed for 1000
		Perfomances computed for 1200
		Perfomances computed for 1400
		Perfomances computed for 1600
Fold no. 2
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
		Perfomances computed for 800
		Perfomances computed for 1000
		Perfomances computed for 1200
		Perfomances computed for 1400
		Perfomances computed for 1600
Fold no. 3
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
		Perfomances computed for 800
		Perfomances computed for 1000
		Perfomances computed for 1200
		Perfomances computed for 1400
		Perfomances computed for 1600
Validation complete, config selected:[[0.1, 2, 0.2], [0.3, 2, 0.3], [0.3, 3, 0.2]]
	R

In [12]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_1)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_1)

Perfomances computed for dictionary settings 1:
	Accuracy: 0.7333333333333333
	Precision: 0.75
	Recall: 0.7105263157894737
Perfomances computed for dictionary settings 2:
	Accuracy: 0.72
	Precision: 0.7297297297297297
	Recall: 0.7105263157894737


In [13]:
w_dict

[{'ds1': {'gaussian': 0.2, 'laplacian': 0.2, 'polynomial': 2},
  'ds2': {'gaussian': 0.3, 'laplacian': 0.2, 'polynomial': 2},
  'ds3': {'gaussian': 0.2, 'laplacian': 0.3, 'polynomial': 3}},
 {'ds1': {'gaussian': 0.7, 'laplacian': 0.5, 'polynomial': 5},
  'ds2': {'gaussian': 0.7, 'laplacian': 0.5, 'polynomial': 5},
  'ds3': {'gaussian': 0.5, 'laplacian': 0.7, 'polynomial': 5}}]

In [14]:
result1.performancesFeatures()

statistics of configuration 1
{'config': {'laplacian': [[0.1, 0.3, 0.3], [0.2, 0.2, 0.3], [0.2, 0.1, 0.3]], 'polynomial': [[2, 2, 3], [2, 3, 3], [2, 2, 3]], 'gaussian': [[0.2, 0.3, 0.2], [0.2, 0.3, 0.2], [0.3, 0.2, 0.3]]}, 'CA': (0.19525793939223526, 6.932870304139932e-05), 'Accuracy': (0.847953216374269, 0.0019151191819705203), 'Precision': (0.8869047619047619, 0.005409108087679518), 'Recall': (0.7976190476190476, 0.0002834467120181403), 'eta': (array([ 2.56656632,  0.02444723, -2.02177008,  1.35893211,  0.00758872,
       -0.6535233 , -1.16048744,  0.00289034,  0.66268454]), array([4.41181704e-01, 3.05620146e-05, 1.35680783e+00, 5.23515163e-01,
       4.26102136e-05, 1.16606965e+00, 5.35292686e-01, 7.82803781e-07,
       2.31652331e-01]))}
statistics of configuration 2
{'config': {'laplacian': [[0.5, 0.5, 0.7], [0.5, 0.5, 0.5], [0.5, 0.5, 0.7]], 'polynomial': [[5, 5, 5], [5, 5, 5], [5, 5, 5]], 'gaussian': [[0.5, 0.5, 0.5], [0.7, 0.7, 0.7], [0.7, 0.7, 0.5]]}, 'CA': (0.1311746696104923

In [15]:
result1 = sampler.sample(kernel_type_2, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)


1 split out of 3 ...
	Working on config 1 of 2: {'sigmoid': [0.1, 0.2, 0.3], 'polynomial': [2, 3], 'gaussian': [0.2, 0.3]}
Fold no. 1
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
		Perfomances computed for 800
		Perfomances computed for 1000
		Perfomances computed for 1200
		Perfomances computed for 1400
		Perfomances computed for 1600
Fold no. 2
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
		Perfomances computed for 800
		Perfomances computed for 1000
		Perfomances computed for 1200
		Perfomances computed for 1400
		Perfomances computed for 1600
Fold no. 3
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
		Perfomances computed for 800
		Perfomances computed for 1000
		Perfomances computed for 1200
		Perfomances computed for 1400
		Perfomances computed for 1600
Validation complete, config selected:[[0.1, 2, 0.2], [0.3, 2, 0.2], [0.3, 3, 0.3]]
	Res

In [16]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_2)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_2)

Perfomances computed for dictionary settings 1:
	Accuracy: 0.7333333333333333
	Precision: 0.75
	Recall: 0.7105263157894737
Perfomances computed for dictionary settings 2:
	Accuracy: 0.7466666666666667
	Precision: 0.7435897435897436
	Recall: 0.7631578947368421


In [17]:
w_dict

[{'ds1': {'gaussian': 0.2, 'polynomial': 2, 'sigmoid': 0.1},
  'ds2': {'gaussian': 0.2, 'polynomial': 2, 'sigmoid': 0.3},
  'ds3': {'gaussian': 0.3, 'polynomial': 3, 'sigmoid': 0.3}},
 {'ds1': {'gaussian': 0.5, 'polynomial': 5, 'sigmoid': 0.7},
  'ds2': {'gaussian': 0.5, 'polynomial': 8, 'sigmoid': 0.7},
  'ds3': {'gaussian': 0.5, 'polynomial': 5, 'sigmoid': 0.7}}]

In [18]:
result1.performancesFeatures()

statistics of configuration 1
{'config': {'sigmoid': [[0.1, 0.3, 0.3], [0.3, 0.3, 0.2], [0.1, 0.2, 0.3]], 'polynomial': [[2, 2, 3], [2, 2, 3], [2, 2, 3]], 'gaussian': [[0.2, 0.2, 0.3], [0.2, 0.2, 0.3], [0.2, 0.3, 0.2]]}, 'CA': (0.2035696566385433, 0.0001736630882767848), 'Accuracy': (0.7719298245614036, 0.00020519134092541295), 'Precision': (0.7528735632183908, 0.00020169991632536322), 'Recall': (0.7976190476190476, 0.0002834467120181403), 'eta': (array([ 0.16123371,  0.0269687 ,  0.53323848,  0.07075751,  0.00717553,
        0.31751098, -0.02897168,  0.0037281 , -0.485689  ]), array([1.44461839e-03, 8.27985314e-06, 2.36425101e-02, 1.65608952e-04,
       1.77546179e-06, 2.60483258e-01, 8.19066431e-05, 3.71391773e-07,
       1.68760357e-01]))}
statistics of configuration 2
{'config': {'sigmoid': [[0.7, 0.7, 0.7], [0.7, 0.7, 0.7], [0.7, 0.7, 0.7]], 'polynomial': [[5, 8, 5], [5, 8, 5], [5, 8, 5]], 'gaussian': [[0.5, 0.5, 0.7], [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]]}, 'CA': (0.17758415405772707

In [26]:
result13 = sampler.sample(kernel_type_3, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)


1 split out of 3 ...
	Working on config 1 of 2: {'linear': [0.5], 'polynomial': [2, 3, 7], 'gaussian': [0.1, 0.5, 0.7]}
Fold no. 1
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
Fold no. 2
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
Fold no. 3
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
Validation complete, config selected:[[0.5, 2, 0.1], [0.5, 3, 0.1], [0.5, 7, 0.1]]
	Result of 1:
CA: 0.20582091129760163
Accuracy: 0.7719298245614035
Precision: 0.8
Recall: 0.7142857142857143
[linear:0.5, polynomial:2, gaussian:0.1, ]
[linear:0.5, polynomial:3, gaussian:0.1, ]
[linear:0.5, polynomial:7, gaussian:0.1, ]

eta vector: [ 0.73151087  2.09998511 -0.04762805  1.2954489  -0.69356657  0.91887983
 -0.27627122 -0.15151821  0.29447802]


	Completed in 1.15 minutes
	Working on config 2 of 2: {'linear': [0.2], 'polynomial': [4, 5, 8], 'gaussian': [0.7, 1]}
F

In [27]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_3)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_3)

Perfomances computed for dictionary settings 1:
	Accuracy: 0.7333333333333333
	Precision: 0.75
	Recall: 0.7105263157894737
Perfomances computed for dictionary settings 2:
	Accuracy: 0.7466666666666667
	Precision: 0.7567567567567568
	Recall: 0.7368421052631579


In [28]:
w_dict

[{'ds1': {'gaussian': 0.2, 'linear': 0.1, 'polynomial': 2},
  'ds2': {'gaussian': 0.2, 'linear': 0.3, 'polynomial': 2},
  'ds3': {'gaussian': 0.3, 'linear': 0.3, 'polynomial': 3}},
 {'ds1': {'gaussian': 0.5, 'linear': 0.7, 'polynomial': 5},
  'ds2': {'gaussian': 0.5, 'linear': 0.7, 'polynomial': 8},
  'ds3': {'gaussian': 0.5, 'linear': 0.7, 'polynomial': 5}}]

In [29]:
result13.performancesFeatures()

statistics of configuration 1
{'config': {'linear': [[0.5, 0.5, 0.5], [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]], 'polynomial': [[2, 3, 7], [2, 7, 7], [2, 2, 3]], 'gaussian': [[0.1, 0.1, 0.1], [0.1, 0.1, 0.1], [0.1, 0.1, 0.5]]}, 'CA': (0.20880977974364445, 4.018590369068768e-05), 'Accuracy': (0.7660818713450293, 0.0025306932047467617), 'Precision': (0.7569683908045978, 0.0023241284350640763), 'Recall': (0.7738095238095238, 0.007086167800453517), 'eta': (array([ 1.01734446,  1.74985156,  0.30129647,  0.54971543,  0.34317211,
        0.55809182, -0.07616287, -0.12879731, -0.00393794]), array([1.06080523e-01, 1.08076945e-01, 6.39386734e-02, 7.33180779e-01,
       1.45749870e+00, 7.64530344e-02, 2.24282604e-02, 9.50372996e-04,
       7.22143717e-02]))}
statistics of configuration 2
{'config': {'linear': [[0.2, 0.2, 0.2], [0.2, 0.2, 0.2], [0.2, 0.2, 0.2]], 'polynomial': [[4, 8, 4], [8, 8, 5], [8, 4, 4]], 'gaussian': [[1, 0.7, 0.7], [0.7, 0.7, 0.7], [0.7, 1, 0.7]]}, 'CA': (0.1971450015415741, 8.33187

## Kernel normalization

In [30]:
sampler = ms.mySampler(n_splits=3, test_size=.25, normalize_kernels = True)

In [ ]:
result1 = sampler.sample(kernel_type_1, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)


1 split out of 3 ...
	Working on config 1 of 2: {'laplacian': [0.1, 0.2, 0.3], 'polynomial': [2, 3], 'gaussian': [0.2, 0.3]}
Fold no. 1
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
		Perfomances computed for 800
		Perfomances computed for 1000
		Perfomances computed for 1200
		Perfomances computed for 1400
		Perfomances computed for 1600
Fold no. 2
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
		Perfomances computed for 800
		Perfomances computed for 1000
		Perfomances computed for 1200
		Perfomances computed for 1400
		Perfomances computed for 1600
Fold no. 3
		Perfomances computed for 200
		Perfomances computed for 400
		Perfomances computed for 600
		Perfomances computed for 800
		Perfomances computed for 1000
		Perfomances computed for 1200
		Perfomances computed for 1400
		Perfomances computed for 1600
Validation complete, config selected:[[0.2, 2, 0.2], [0.1, 2, 0.3], [0.3, 2, 0.2]]
	R

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_1)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_1)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result1 = sampler.sample(kernel_type_2, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_2)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_2)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result13 = sampler.sample(kernel_type_3, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_3)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_3)

In [ ]:
w_dict

In [ ]:
result13.performancesFeatures()

## Normalized data

In [ ]:
sampler = ms.mySampler(n_splits=3, test_size=.25, normalizing = True)

In [ ]:
result1 = sampler.sample(kernel_type_1, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_1)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_1)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result1 = sampler.sample(kernel_type_2, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_2)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_2)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result13 = sampler.sample(kernel_type_3, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_3)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_3)

In [ ]:
w_dict

In [ ]:
result13.performancesFeatures()

## Origin Data Centering

In [ ]:
sampler = ms.mySampler(n_splits=3, test_size=.25, centering = True)

In [ ]:
result1 = sampler.sample(kernel_type_1, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_1)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_1)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result1 = sampler.sample(kernel_type_2, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_2)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_2)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result13 = sampler.sample(kernel_type_3, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_3)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_3)

In [ ]:
w_dict

In [ ]:
result13.performancesFeatures()

## Origin Data  Centering and Normalization

In [ ]:
sampler = ms.mySampler(n_splits=3, test_size=.25, centering = True, normalizing = True)

In [ ]:
result1 = sampler.sample(kernel_type_1, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_1)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_1)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result1 = sampler.sample(kernel_type_2, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_2)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_2)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result13 = sampler.sample(kernel_type_3, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_3)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_3)

In [ ]:
w_dict

In [ ]:
result13.performancesFeatures()

## Normalization, Kernel Normalization

In [ ]:
sampler = ms.mySampler(n_splits=3, test_size=.25, normalizing = True, normalize_kernels = True)

In [ ]:
result1 = sampler.sample(kernel_type_1, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_1)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_1)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result1 = sampler.sample(kernel_type_2, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_2)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_2)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result13 = sampler.sample(kernel_type_3, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_3)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_3)

In [ ]:
w_dict

In [ ]:
result13.performancesFeatures()

## Centering, Kernel Normalization

In [ ]:
sampler = ms.mySampler(n_splits=3, test_size=.25, centering = True, normalize_kernels = True)

In [ ]:
result1 = sampler.sample(kernel_type_1, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_1)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_1)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result1 = sampler.sample(kernel_type_2, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_2)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_2)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result13 = sampler.sample(kernel_type_3, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_3)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_3)

In [ ]:
w_dict

In [ ]:
result13.performancesFeatures()

## Centering, Normalization, Kernel Normalization

In [ ]:
sampler = ms.mySampler(n_splits=3, test_size=.25, centering = True, normalizing = True, normalize_kernels = True)

In [ ]:
result1 = sampler.sample(kernel_type_1, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_1)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_1)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result1 = sampler.sample(kernel_type_2, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_2)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_2)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result13 = sampler.sample(kernel_type_3, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_3)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_3)

In [ ]:
w_dict

In [ ]:
result13.performancesFeatures()

## L2 Penalty, Centering, K-Normalization

In [ ]:
sampler = ms.mySampler(n_splits=3, test_size=0.25, lamb = 0.7, centering = True, normalize_kernels = False)

In [ ]:
result1 = sampler.sample(kernel_type_1, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_1)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_1)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result1 = sampler.sample(kernel_type_2, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_2)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_2)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result13 = sampler.sample(kernel_type_3, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_3)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_3)

In [ ]:
w_dict

In [ ]:
result13.performancesFeatures()

## Sparsity in eta, Centering, K-Normalization

In [ ]:
sampler = ms.mySampler(n_splits=3, test_size=0.25, lamb = 0.7, centering = True, normalize_kernels = False)

In [ ]:
result1 = sampler.sample(kernel_type_1, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_1)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_1)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result1 = sampler.sample(kernel_type_2, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_2)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_2)

In [ ]:
w_dict

In [ ]:
result1.performancesFeatures()

In [ ]:
result13 = sampler.sample(kernel_type_3, estimator, ds_list, y_c_, valid_fold = 3, verbose=True)

In [ ]:
w_dict, w_list = result1.votingOverCA(ds_names, kernel_names_3)
ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_3)

In [ ]:
w_dict

In [ ]:
result13.performancesFeatures()